# 大模型训练实验

实验目标：

在 Hugging Face 上加载一个**真实文本分类数据集**（SST-2），使用 Hugging Face 的 **Tokenizer** 做分词，训练一个小模型进行情感分类（positive / negative）。

目录：

1. 用 PyTorch 从零训练 Tiny Transformer 模型
2. 改进 Tiny Transformer 训练效果
3. 在预训练模型 distilbert-base-uncased 基础上做微调
4. 提高预训练模型的微调速度

运行环境：

* macOS Apple Silicon (M1–M4)
* PyTorch ≥ 2.2（支持 MPS）
* `transformers`、`datasets` 来自 Hugging Face

安装依赖：

```bash
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
pip install transformers datasets
```

# 一、用 PyTorch 从零训练 Tiny Transformer 模型

训练一个 非常小的 Transformer encoder 做序列分类（synthetic toy），展示 embedding、positional encoding、TransformerEncoder、训练循环与 MPS/CPU 互换。模型很小，能在 M4/macbook 上快速跑完。

### 1. 导入依赖与设定设备

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
from tqdm.notebook import tqdm

# 选择设备
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


### 2. 加载真实数据集（SST-2）

In [2]:
# 加载 Stanford Sentiment Treebank (SST-2)
dataset = load_dataset("glue", "sst2")

print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


### 3. 加载 Hugging Face 分词器

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

max_len = 64

def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=max_len
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

### 4. 定义 Tiny Transformer 模型

In [4]:
class TinyTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=2, dim_feedforward=256, num_classes=2, max_len=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, num_classes)
        )

    def forward(self, input_ids, attention_mask=None):
        x = self.embedding(input_ids) + self.pos_embedding[:, :input_ids.size(1)]
        x = x.transpose(0, 1)  # [seq, batch, d_model]
        x = self.transformer(x)  # [seq, batch, d_model]
        x = x.mean(dim=0)  # pool over sequence
        logits = self.fc(x)
        return logits


### 5️⃣ 构建 DataLoader

In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

### 5. 训练与验证循环

In [6]:
model = TinyTransformerClassifier(vocab_size=len(tokenizer), num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

def evaluate(model, loader):
    model.eval()
    correct, total, total_loss = 0, 0, 0
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * input_ids.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / total, correct / total

EPOCHS = 3
for epoch in range(EPOCHS):
    model.train()
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

    val_loss, val_acc = evaluate(model, val_loader)
    print(f"Validation Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

/Users/huhao/.pyenv/versions/3.11.9/envs/qlib/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1:   0%|          | 0/1053 [00:00<?, ?it/s]

Validation Loss: 0.5078, Acc: 0.7557


Epoch 2:   0%|          | 0/1053 [00:00<?, ?it/s]

Validation Loss: 0.5271, Acc: 0.7672


Epoch 3:   0%|          | 0/1053 [00:00<?, ?it/s]

Validation Loss: 0.5921, Acc: 0.7764


### 6. 测试样例

In [7]:
test_sentence = "This movie was surprisingly good!"
inputs = tokenizer(test_sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=max_len).to(device)
logits = model(inputs["input_ids"])
pred = torch.argmax(logits, dim=1).item()
print("Sentence:", test_sentence)
print("Predicted sentiment:", "positive" if pred == 1 else "negative")

Sentence: This movie was surprisingly good!
Predicted sentiment: positive


### 实验说明

| 部分        | 内容                                                                       |
| --------- | ------------------------------------------------------------------------ |
| 数据集       | [GLUE SST-2](https://huggingface.co/datasets/glue/viewer/sst2)（电影评论情感分类） |
| Tokenizer | `distilbert-base-uncased`（真实 BERT 词表）                                    |
| 模型        | 自定义 Tiny Transformer（非预训练）                                               |
| 任务        | 文本 → 二分类（情感极性）                                                           |
| 加速        | 自动检测 MPS (Metal GPU)                                                     |
| 训练时长      | M4 Mac 上约 1–2 分钟即可跑完几轮                                                   |

# 二、改进 Tiny Transformer 训练效果

### 改进目标

| 改进点                                   | 目的          |
| ------------------------------------- | ----------- |
| 增大模型容量（`d_model=256`, `num_layers=3`） | 提升表达能力      |
| 使用 Dropout（防止过拟合）                     | 提升泛化        |
| 学习率衰减调度器                              | 更稳定收敛       |
| 增加训练轮数至 8                             | 充分训练        |
| 自动保存最优模型（验证集最高 acc）                   | 保存最佳参数      |
| 增加 LayerNorm 稳定性                      | 减少梯度爆炸/收敛震荡 |


### 1. 模型定义部分

In [8]:
class TinyTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=4, num_layers=3, dim_feedforward=512,
                 num_classes=2, max_len=64, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_len, d_model))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='gelu',
            batch_first=True,  # 可直接使用 batch 维在前的输入
            norm_first=True,   # LayerNorm 前置稳定训练
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.layernorm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )

    def forward(self, input_ids):
        x = self.embedding(input_ids) + self.pos_embedding[:, :input_ids.size(1)]
        x = self.dropout(x)
        x = self.transformer(x)
        x = self.layernorm(x)
        x = x.mean(dim=1)  # mean pooling
        logits = self.fc(x)
        return logits

### 2. 训练循环部分（含调度器与保存）

In [9]:
import torch
import numpy as np
from tqdm.notebook import tqdm

model = TinyTransformerClassifier(vocab_size=len(tokenizer), num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)

best_val_acc = 0.0
EPOCHS = 8

def evaluate(model, loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * input_ids.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / total, correct / total


for epoch in range(1, EPOCHS + 1):
    model.train()
    loop = tqdm(train_loader, desc=f"Epoch {epoch}")
    running_loss = 0.0
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # 防止梯度爆炸
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    scheduler.step()
    val_loss, val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch}: Val Loss={val_loss:.4f}  Acc={val_acc:.4f}")

    # 自动保存最优模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_tiny_transformer.pt")
        print(f"✅ Saved new best model at epoch {epoch} (acc={val_acc:.4f})")

print("Training complete! Best validation accuracy:", best_val_acc)

/Users/huhao/.pyenv/versions/3.11.9/envs/qlib/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Epoch 1:   0%|          | 0/1053 [00:00<?, ?it/s]

Epoch 1: Val Loss=0.5199  Acc=0.7603
✅ Saved new best model at epoch 1 (acc=0.7603)


Epoch 2:   0%|          | 0/1053 [00:00<?, ?it/s]

Epoch 2: Val Loss=0.5300  Acc=0.7833
✅ Saved new best model at epoch 2 (acc=0.7833)


...

Epoch 6: Val Loss=0.5956  Acc=0.8062


Epoch 7:   0%|          | 0/1053 [00:00<?, ?it/s]

Epoch 7: Val Loss=0.6262  Acc=0.8062


Epoch 8:   0%|          | 0/1053 [00:00<?, ?it/s]

Epoch 8: Val Loss=0.6660  Acc=0.8016
Training complete! Best validation accuracy: 0.8073394495412844


### 3. 加载模型进行测试预测

In [10]:
# 加载最优模型
model2 = TinyTransformerClassifier(vocab_size=len(tokenizer), num_classes=2).to(device)
model2.load_state_dict(torch.load("best_tiny_transformer.pt", map_location=device))
model2.eval()

# 测试一句话
sentence = "I really loved this movie, it was brilliant!"
inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=64)
inputs = {k: v.to(device) for k, v in inputs.items()}
logits = model2(inputs["input_ids"])
pred = torch.argmax(logits, dim=1).item()
print(f"Sentence: {sentence}")
print("Predicted sentiment:", "positive" if pred == 1 else "negative")

Sentence: I really loved this movie, it was brilliant!
Predicted sentiment: positive


# 三、在预训练模型 distilbert-base-uncased 基础上做微调

直接用 Hugging Face 上的 **预训练 BERT/DistilBERT** 模型进行微调。

### 1. 为什么使用预训练模型（BERT / DistilBERT）？

这是 **效果 vs 速度** 的核心权衡。它们是“通用语言知识库”

* BERT / DistilBERT 不是随机初始化的网络，而是在**超大语料**（Wikipedia、BooksCorpus 等）上预训练过的。
* 它们已经**学习了英语语法、词义、上下文关系**。
* 因此，只需要**微调（fine-tune）几轮**，就能在情感分析、问答、分类等任务上取得非常高的准确率。

> 举例：
>
> * 你的 Tiny Transformer（从零训练）靠随机初始化，需要学习“good”=正面、“bad”=负面等全部知识。
> * BERT 已经“知道”这些词义，只要轻微调整最后几层参数。

💡 所以我们选择 BERT / DistilBERT，是因为：

* 它们在大量语料上预训练；
* 微调时效果极好；
* Hugging Face 提供了现成的权重和分词器；
* 对多数 NLP 任务都是标准 baseline。

禁用分词器的并发功能

In [11]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 2. 定义分词器，训练参数，训练器，开始训练

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("glue", "sst2")

def preprocess(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=64)

tokenized = dataset.map(preprocess, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

model3 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    max_steps=200,             # 训练太慢，我们训练 200 个 step 就停止
    output_dir="./results",
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    dataloader_pin_memory=False,
)

trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    processing_class=tokenizer,
)

trainer.train()
trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
0,No log,0.361624


{'eval_loss': 0.36162370443344116,
 'eval_runtime': 2.2198,
 'eval_samples_per_second': 392.834,
 'eval_steps_per_second': 12.614,
 'epoch': 0.047505938242280284}

训练太慢，我们提前停止

# 四、提高预训练模型的微调速度

### 1. 为什么训练会变慢？

1. 模型参数量巨大

| 模型                     | 参数量     | 说明                               |
| ---------------------- | ------- | -------------------------------- |
| Tiny Transformer（你自写的） | 约 1–3 M | 小型网络，几毫秒每 batch                  |
| DistilBERT             | ~66 M   | 已经是轻量版                           |
| BERT base              | ~110 M  | 12 层 Transformer、768 hidden size |
| RoBERTa / GPT          | 数亿以上    | 研究级模型                            |

→ 这就意味着每次前向 / 反向传播的矩阵乘法开销远远大于你的小模型。

2。 预训练模型的计算图更复杂

* 它包含 **多头注意力、层归一化、残差连接、dropout** 等结构；
* 同时还要计算 attention mask；
* 而你的小模型是简化版（无复杂 mask 操作），显然快很多。

3. macOS 上 GPU (MPS) 尚未完全优化 Transformer

* MPS（Metal Performance Shaders）比 CUDA 慢约 2–4 倍（尤其在多头注意力上）；
* BERT 微调在 macOS 上通常比在 CUDA 上慢几倍；
* 对 M4 芯片来说，虽然芯片性能强，但 PyTorch MPS 后端还未达到 CUDA 的成熟度；
* 若你的 batch size 较大（例如 32），显存/内存拷贝也会成为瓶颈。

👉 简言之：**慢的根本原因是模型大 + MPS 性能限制。**

### 2. 解决思路：如何加快训练？

| 方法                                       | 说明                   | 建议                                                   |
| ---------------------------------------- | -------------------- | ---------------------------------------------------- |
| ✅ **用 DistilBERT 替代 BERT**               | 已是轻量化版本，参数减半         | 推荐使用 `"distilbert-base-uncased"`（你已经在用）              |
| ⚙️ **降低 max_length**                     | 减少序列长度（64→32）减少计算量   | `tokenizer(..., max_length=32)`                      |
| 🔢 **减少 batch_size**                     | 减少显存压力，提升每步速度        | `batch_size=8` 或 `16`                                |
| ⏳ **冻结底层参数**                             | 微调时只训练顶部分类层          | 见下代码示例                                               |
| 💾 **混合精度训练 (fp16)**                     | 在 CUDA 可用，MPS 暂不完全支持 | 暂时不推荐 MPS 上用                                         |
| 🧩 **DistilBERT Tiny / ALBERT / MiniLM** | 更小模型，快 3–5 倍         | `"prajjwal1/bert-tiny"`, `"distilbert-base-uncased"` |

---

示例：只微调分类头（冻结底层层）

你可以让 BERT 的大部分层冻结，只训练最后几层，这样快很多，然后再用 `Trainer` 训练，速度通常会提升 5–10 倍，效果略低（准确率仍可 >85%）。

In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

model4 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', max_length=32)

def print_trainable_parameters(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable}/{total} ({trainable/total:.2%})")

print_trainable_parameters(model4)

# 冻结除最后一层以外的参数
for name, param in model4.named_parameters():
    if "classifier" not in name:   # 只训练分类层
        param.requires_grad = False

print_trainable_parameters(model4)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 66955010/66955010 (100.00%)
Trainable parameters: 592130/66955010 (0.88%)


### 3. 开始微调

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    dataloader_pin_memory=False,
)

from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {"accuracy": acc, "f1": f1}


trainer = Trainer(
    model=model4,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.366200,0.390276,0.826835,0.824214
2,0.373500,0.388247,0.832569,0.833713
3,0.357200,0.385876,0.832569,0.834842


{'eval_loss': 0.3858757019042969,
 'eval_accuracy': 0.8325688073394495,
 'eval_f1': 0.834841628959276,
 'eval_runtime': 2.7555,
 'eval_samples_per_second': 316.459,
 'eval_steps_per_second': 39.557,
 'epoch': 3.0}

In [15]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.3858757019042969, 'eval_accuracy': 0.8325688073394495, 'eval_f1': 0.834841628959276, 'eval_runtime': 2.6871, 'eval_samples_per_second': 324.508, 'eval_steps_per_second': 40.563, 'epoch': 3.0}


### 4. 测试微调效果

In [16]:
texts = [
    "This movie was absolutely wonderful!",
    "The plot was terrible and the acting was worse.",
    "It's okay, not great but not bad either."
]

model4.to("mps")
inputs = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors="pt").to("mps")
model4.eval()
with torch.no_grad():
    outputs = model4(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)


for text, pred in zip(texts, predictions):
    label = "positive" if pred.item() == 1 else "negative"
    print(f"{text}\n → Predicted sentiment: {label}\n")

This movie was absolutely wonderful!
 → Predicted sentiment: positive

The plot was terrible and the acting was worse.
 → Predicted sentiment: negative

It's okay, not great but not bad either.
 → Predicted sentiment: positive

